In [ ]:
import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.utils import plot_model
import time
import pickle
from keras.utils import to_categorical
from scipy.sparse import csr_matrix

In [ ]:
model = VGG16()

In [ ]:
model.summary()

In [ ]:
mkdir images

In [ ]:
plot_model(model, "images/vgg16.png", show_shapes=True)

![](images/vgg16.png)

In [ ]:
model.layers.pop()
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

In [ ]:
model.summary()

In [ ]:
model.save("../data/photo_processor/model.h5")

In [ ]:
def extract_photo_features(list_file):
    photo_list = pd.read_csv(open(list_file,'r'), encoding='utf-8', engine='c')['photo_id']
    features = dict()
    i = 0
    starttime = time.time()
    for photo in photo_list:
        if i % 500 == 0:
            print("Processed %d photos in %f seconds" % (i, starttime - time.time()))
        i += 1
        fname = "../data/yelp_photos/photos/%s.jpg" % (photo)
        try: 
            img = load_img(fname, target_size = (224,224))
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img)
            feat = model.predict(img, verbose=0)
            features[photo] = feat
        except:
            continue
    return features

In [ ]:
train_features = extract_photo_features("../data/split_lists/train_ids.csv")

In [ ]:
valid_features = extract_photo_features("../data/split_lists/valid_ids.csv")

In [ ]:
mkdir ../data/features

In [ ]:
with open("../data/features/train_features.pkl", "wb") as handle:
    pickle.dump(train_features, handle)

In [ ]:
with open("../data/features/valid_features.pkl", "wb") as handle:
    pickle.dump(valid_features,handle)

In [ ]:
len(valid_features)

In [ ]:
len(train_features)

In [ ]:
to_categorical(range(10), 1000).nbytes

In [ ]:
csr_matrix(to_categorical(range(10), 1000)).nbytes